In [40]:
knitr::opts_chunk$set(echo = TRUE)

# Install the required packages

In [41]:
options(repos = c(CRAN = "https://cran.rstudio.com/"))

if (!require("ggplot2")) install.packages("ggplot2")
if (!require("e1071")) install.packages("e1071")
if (!require("tidyverse")) install.packages("tidyverse")
if (!require("dplyr")) install.packages("dplyr")
if (!require("effsize")) install.packages("effsize")

In [42]:
install.packages("car")
install.packages('cowplot')
install.packages('grid')
install.packages("png")

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

Warning message:
“package ‘grid’ is a base package, and should not be updated”
Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)



In [43]:
library("car")
library('cowplot')
library('grid')
library("png")

library(e1071)
library(effsize)

library(ggplot2)
library(tidyverse)
library(dplyr)
library(knitr)


# Define some global variables

In [55]:
PROVOOST <- "provoost"
OURS <- "ours"

TREATMENTS <- c(PROVOOST,OURS)

# Color mapping for On-Device and Remote using R's native colors
COLOR_MAP <- c(provoost = "coral",
               ours = "lightblue")

PACKET_LOSS <- "avg_packet_loss_percentage"
ENERGY <- "total_energy_consumption_mJ"

METRICS <- c(PACKET_LOSS, ENERGY)

AXIS_LABELS <- list(
  avg_packet_loss_percentage = "Average Packet Loss (%)",
  total_energy_consumption_mJ = "Total Energy Consumption (mJ)"
)

VIOLIN_FOLDER <- "violin_plots"

PLOT_FOLDERS <- c(VIOLIN_FOLDER)

WIDTH = 12
HEIGHT = 12
FONT_MULTIPLIER = 2

# Read the run table file and multiply PACKET_LOSS with 100

In [45]:
# Read the dataset
dataset <- read.csv("./run_table.csv")

print(head(dataset))


           X__run_id X__done adaptation_mode avg_packet_loss_percentage
1 run_0_repetition_0    DONE        provoost                   14.91694
2 run_0_repetition_1    DONE        provoost                   15.26036
3 run_0_repetition_2    DONE        provoost                   15.07233
4 run_0_repetition_3    DONE        provoost                   15.19595
5 run_0_repetition_4    DONE        provoost                   15.37998
6 run_0_repetition_5    DONE        provoost                   15.01255
  total_energy_consumption_mJ
1                    7256.936
2                    7479.798
3                    7078.353
4                    7410.731
5                    7839.350
6                    6918.433


# H1 Analysis.

## Define a Function to Remove Outliers using IQR method (works on multiple columns of a dataset)

In [46]:
# Function to remove outliers using the IQR method
remove_outliers <- function(data, columns) {
  # Initialize filtered data as the original data
  filtered_data <- data

  # Loop over each column and apply IQR outlier filtering
  for (column in columns) {
    Q1 <- quantile(filtered_data[[column]], 0.25, na.rm = TRUE)
    Q3 <- quantile(filtered_data[[column]], 0.75, na.rm = TRUE)
    IQR_value <- Q3 - Q1
    lower_bound <- Q1 - 1.5 * IQR_value
    upper_bound <- Q3 + 1.5 * IQR_value

    # Filter rows where the column values are within bounds
    filtered_data <- filtered_data %>%
      filter(filtered_data[[column]] >= lower_bound & filtered_data[[column]] <= upper_bound)
  }

  return(filtered_data)
}

## Filter dataset into 2 treatments while removing outliers based on `ENERGY` column
- 2 Treatments:
  - `provoost`
  - `ours`

In [47]:
# Create a new dataset object for cleaned data
cleaned_dataset <- data.frame()

# Loop over treatments and apply outlier removal
for (method in TREATMENTS) {
  # Filter the dataset for the specific method
  filtered_data <- dataset %>%
    filter(adaptation_mode == !!method)  # Adjust column name for your dataset

  # Remove outliers using the defined function
  cleaned_subset <- remove_outliers(filtered_data, METRICS)

  # Append the cleaned data to the new dataset
  cleaned_dataset <- bind_rows(cleaned_dataset, cleaned_subset)
}

In [48]:
print(head(cleaned_dataset))

           X__run_id X__done adaptation_mode avg_packet_loss_percentage
1 run_0_repetition_0    DONE        provoost                   14.91694
2 run_0_repetition_1    DONE        provoost                   15.26036
3 run_0_repetition_2    DONE        provoost                   15.07233
4 run_0_repetition_3    DONE        provoost                   15.19595
5 run_0_repetition_4    DONE        provoost                   15.37998
6 run_0_repetition_5    DONE        provoost                   15.01255
  total_energy_consumption_mJ
1                    7256.936
2                    7479.798
3                    7078.353
4                    7410.731
5                    7839.350
6                    6918.433


## Table: Mean, Median, and Standard Deviation (SD) of Energy Consumption and Packet Loss for Different Treatment (Adaptation Strategies)

In [49]:
# Group by adaptation_mode and calculate mean, median, and SD for each metric
summary_table <- cleaned_dataset %>%
  group_by(adaptation_mode) %>%
  summarise(
    mean_packet_loss = mean(.data[[PACKET_LOSS]], na.rm = TRUE),
    median_packet_loss = median(.data[[PACKET_LOSS]], na.rm = TRUE),
    sd_packet_loss = sd(.data[[PACKET_LOSS]], na.rm = TRUE),
    mean_energy = mean(.data[[ENERGY]], na.rm = TRUE),
    median_energy = median(.data[[ENERGY]], na.rm = TRUE),
    sd_energy = sd(.data[[ENERGY]], na.rm = TRUE)
  ) %>%
  mutate(across(where(is.numeric), ~ sprintf("%.2f", .x))) # Format all numeric values to 2 decimal places

# Print the summary table using knitr for a nicely formatted table
kable(summary_table,
      caption = "Mean, Median, and Standard Deviation (SD) of Energy Consumption and Packet Loss for Different Treatment (Adaptation Strategies)")



Table: Mean, Median, and Standard Deviation (SD) of Energy Consumption and Packet Loss for Different Treatment (Adaptation Strategies)

|adaptation_mode |mean_packet_loss |median_packet_loss |sd_packet_loss |mean_energy |median_energy |sd_energy |
|:---------------|:----------------|:------------------|:--------------|:-----------|:-------------|:---------|
|provoost        |15.10            |15.11              |0.15           |7307.76     |7330.50       |304.18    |

# Normality Check

## Visualize data for manual normality check

In [60]:
# Remove all folders listed in PLOT_FOLDERS
for (folder in PLOT_FOLDERS) {
  system(paste("rm -rf", folder))  # Remove the folder if it exists
}

# Loop through the plot folders and create them if they don't exist
for (folder in PLOT_FOLDERS) {
  if (!dir.exists(folder)) {
    dir.create(folder)
  }
}


### Violin Plots

In [61]:
violin_plot <- function(dataset, title, x, y, show_legend = TRUE) {
  ggplot(dataset, aes(x = Distribution, y = Values, fill = Distribution)) +
    geom_violin(alpha = 0.5) +
    geom_boxplot(width = 0.045, outlier.shape = NA) +
    scale_fill_manual(values = COLOR_MAP) +  # Use COLOR_MAP for colors
    labs(
      title = title,
      x = x,
      y = y
    ) +
    theme_minimal() +
    theme(
      legend.position = ifelse(show_legend, "right", "none"),  # Control legend display
      legend.title = element_blank(),
      legend.text = element_text(size = rel(FONT_MULTIPLIER)),  # Increase legend text size
      axis.title = element_text(size = rel(FONT_MULTIPLIER)),
      axis.text = element_text(size = rel(FONT_MULTIPLIER)),
      plot.title = element_text(size = rel(1.1 * FONT_MULTIPLIER))
    )
}

In [62]:
generate_plots_for_metric <- function(metric_name) {
  # Assuming combined_data is already in the correct format:
  # 'combined_data' must have columns: adaptation_mode, Values, Metric (avg_packet_loss or total_energy_consumption_mJ)

  # Create the dataset for the specific metric
  plot_data <- cleaned_dataset %>%
    select(adaptation_mode, all_of(metric_name)) %>%
    gather(key = "Metric", value = "Values", all_of(metric_name)) %>%
    mutate(Distribution = factor(adaptation_mode, levels = TREATMENTS))

  # Create the violin plot title based on the metric
  plot_title_violin <- paste0("Violin Plot for ", AXIS_LABELS[[metric_name]])

  # Generate the violin plot object
  violin_plot_obj <- violin_plot(
    dataset = plot_data,
    title = plot_title_violin,
    x = "Adaptation Strategy",
    y = AXIS_LABELS[[metric_name]],
    show_legend = FALSE
  )

  # Save the violin plot in the specified folder
  ggsave(
    filename = file.path(VIOLIN_FOLDER, paste0("violin_plot_", metric_name, ".png")),
    plot = violin_plot_obj,
    width = WIDTH,
    height = HEIGHT
  )
}

In [63]:
# Loop through each metric and generate plots
for (metric_name in METRICS) {
  generate_plots_for_metric(metric_name)
}

# [Only on Google Colab] Download the Visualization Folders

In [ ]:
zip("all_folders.zip", files = unlist(lapply(PLOT_FOLDERS, function(folder) {
  list.files(folder, full.names = TRUE, recursive = TRUE)
})))



In [ ]:
# Then, manually download the exported .zip